In [1]:
# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
from sqlite3 import dbapi2 as sq3
import os

PATHSTART="."

def get_db(dbfile):
    sqlite_db = sq3.connect(os.path.join(PATHSTART, dbfile))
    return sqlite_db

def init_db(dbfile, schema):
    db = get_db(dbfile)
    db.cursor().executescript(schema)
    db.commit()
    return db

In [3]:
ourschema="""
DROP TABLE IF EXISTS "candidates";
DROP TABLE IF EXISTS "contributors";
CREATE TABLE "candidates" (
    "id" INTEGER PRIMARY KEY  NOT NULL ,
    "first_name" VARCHAR,
    "last_name" VARCHAR,
    "middle_name" VARCHAR,
    "party" VARCHAR NOT NULL
);
CREATE TABLE "contributors" (
    "id" INTEGER PRIMARY KEY  AUTOINCREMENT  NOT NULL,
    "last_name" VARCHAR,
    "first_name" VARCHAR,
    "middle_name" VARCHAR,
    "street_1" VARCHAR,
    "street_2" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "zip" VARCHAR,
    "amount" INTEGER,
    "date" DATETIME,
    "candidate_id" INTEGER NOT NULL,
    FOREIGN KEY(candidate_id) REFERENCES candidates(id)
);
"""

In [4]:
dfcand = pd.read_csv('http://dl.dropboxusercontent.com/u/75194/candidates.txt', sep = '|')
dfid = pd.read_csv('http://dl.dropboxusercontent.com/u/75194/contributors_with_candidate_id.txt', sep = '|')
del dfid['id']

In [5]:
db = init_db('cand.db', ourschema)
dfcand.to_sql('candidates', db, if_exists = 'append', index = False)
dfid.to_sql('contributors', db, if_exists = 'append', index = False)

In [6]:
db = get_db('cand.db')
def make_query(sel):
    c = db.cursor().execute(sel)
    return c.fetchall()

make_query('SELECT * FROM candidates')
#make_query('SELECT * FROM contributors')

[(16, 'Mike', 'Huckabee', None, 'R'),
 (20, 'Barack', 'Obama', None, 'D'),
 (22, 'Rudolph', 'Giuliani', None, 'R'),
 (24, 'Mike', 'Gravel', None, 'D'),
 (26, 'John', 'Edwards', None, 'D'),
 (29, 'Bill', 'Richardson', None, 'D'),
 (30, 'Duncan', 'Hunter', None, 'R'),
 (31, 'Dennis', 'Kucinich', None, 'D'),
 (32, 'Ron', 'Paul', None, 'R'),
 (33, 'Joseph', 'Biden', None, 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', None, 'R'),
 (36, 'Samuel', 'Brownback', None, 'R'),
 (37, 'John', 'McCain', None, 'R'),
 (38, 'Tom', 'Tancredo', None, 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

#### Or populate with SQL insert

In [32]:
ins="""
INSERT INTO candidates (id, first_name, last_name, middle_name, party) \
    VALUES (?,?,?,?,?);
"""

with open ("cand.txt") as fd:
    slines = [l.strip().split('|') for l in fd.readlines()]
    for line in slines[1:]:
        a, b, c, d, e = line
        value_insert = (int(a), b, c, d, e)
        print (value_insert)
        
#       db.cursor().execute(ins, valstoinsert)

print(slines)


(33, 'Joseph', 'Biden', '', 'D')
(36, 'Samuel', 'Brownback', '', 'R')
(34, 'Hillary', 'Clinton', 'R.', 'D')
(39, 'Christopher', 'Dodd', 'J.', 'D')
(26, 'John', 'Edwards', '', 'D')
(22, 'Rudolph', 'Giuliani', '', 'R')
(24, 'Mike', 'Gravel', '', 'D')
(16, 'Mike', 'Huckabee', '', 'R')
(30, 'Duncan', 'Hunter', '', 'R')
(31, 'Dennis', 'Kucinich', '', 'D')
(37, 'John', 'McCain', '', 'R')
(20, 'Barack', 'Obama', '', 'D')
(32, 'Ron', 'Paul', '', 'R')
(29, 'Bill', 'Richardson', '', 'D')
(35, 'Mitt', 'Romney', '', 'R')
(38, 'Tom', 'Tancredo', '', 'R')
(41, 'Fred', 'Thompson', 'D.', 'R')
[['id', 'first_name', 'last_name', 'middle_name', 'party'], ['33', 'Joseph', 'Biden', '', 'D'], ['36', 'Samuel', 'Brownback', '', 'R'], ['34', 'Hillary', 'Clinton', 'R.', 'D'], ['39', 'Christopher', 'Dodd', 'J.', 'D'], ['26', 'John', 'Edwards', '', 'D'], ['22', 'Rudolph', 'Giuliani', '', 'R'], ['24', 'Mike', 'Gravel', '', 'D'], ['16', 'Mike', 'Huckabee', '', 'R'], ['30', 'Duncan', 'Hunter', '', 'R'], ['31', 'Denn

## Query with SQL + Pandas

In [7]:
cont_cols = [e[1] for e in make_query('PRAGMA table_info(contributors)')]
cont_cols

#for e in make_query("PRAGMA table_info(contributors);"):
#   print (e[1])

['id',
 'last_name',
 'first_name',
 'middle_name',
 'street_1',
 'street_2',
 'city',
 'state',
 'zip',
 'amount',
 'date',
 'candidate_id']

In [8]:
out = make_query("SELECT * FROM contributors WHERE state = 'VA' AND amount < 400")
print (out)

[(28, 'Buckheit', 'Bruce', None, '8904 KAREN DR', None, 'FAIRFAX', 'VA', '220312731', 100, '2007-09-19', 20), (78, 'Ranganath', 'Anoop', None, '2507 Willard Drive', None, 'Charlottesville', 'VA', '22903', -100, '2008-04-21', 32), (89, 'Perreault', 'Louise', None, '503 Brockridge Hunt Drive', None, 'Hampton', 'VA', '23666', -34.08, '2008-04-21', 32), (146, 'ABDELLA', 'THOMAS', 'M.', '4231 MONUMENT WALL WAY #340', None, 'FAIRFAX', 'VA', '220308440', 50, '2007-09-30', 35)]


In [9]:
def make_frame(list_of_tuples, legend = cont_cols):
    frame_list = []
    for i, cols in enumerate(legend):
        frame_list.append((cols, [e[i] for e in list_of_tuples]))
    return pd.DataFrame.from_items(frame_list)
    

make_frame(out)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20
1,78,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32
2,89,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32
3,146,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,None,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [44]:
dfid[dfid.state.isnull()]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
125,BOURNE,TRAVIS,NaN,LAGE KAART 77,NaN,BRASSCHATT,NaN,2930,-500.0,2008-11-20,35


In [46]:
out = make_query("SELECT * FROM contributors WHERE state IS NOT NULL")
make_frame(out).shape

(174, 12)

In [50]:
out = make_query("SELECT * FROM contributors WHERE state IN ('VA', 'MA')")
make_frame(out).head()
#out = make_query("SELECT * FROM contributors WHERE state BETWEEN 10 AND 50")

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500.00,2007-06-30,16
1,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20
2,78,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32
3,79,Nystrom,Michael,A,93A Fairmont Street,None,Arlington,MA,2474,-503.00,2008-04-21,32
4,89,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32


In [55]:
dfid[(dfid['amount']>10) & (dfid['amount']<50)]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
26,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,25.0,2007-08-16,20
49,Harrison,Ryan,NaN,2247 3rd St,NaN,La Verne,CA,917504918,25.0,2007-07-26,20
101,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25.0,2008-02-26,34
140,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-09-17,35
143,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-08-06,35
144,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-07-10,35
158,ABBO,PAULINE,MORENCY,10720 JACOB LANE,NaN,WHITE LAKE,MI,483862274,35.0,2008-01-07,37
160,ABAIR,PETER,NaN,40 EVANS STREET,NaN,WATERTOWN,MA,24722150,25.0,2008-01-09,37


## Sort with SQL + Pandas

In [58]:
dfid.sort('amount').head() # sort in ascending order
dfid.sort('amount', ascending = False).head() # sort in descending order

C:\Users\Seuphor\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':
C:\Users\Seuphor\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.0,2007-08-14,20
159,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,NaN,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
15,Anthony,John,NaN,211 Long Island Drive,NaN,Hot Springs,AR,71913,2300.0,2007-06-12,16
33,Buck,Blaine,M,45 Eaton Ave,NaN,Camden,ME,48431752,2300.0,2007-09-30,20
28,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,2300.0,2007-08-14,20


In [61]:
out = make_query("SELECT * FROM contributors ORDER BY amount")
make_frame(out).head()

out = make_query("SELECT * FROM contributors ORDER BY amount DESC")
make_frame(out).head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,31,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,4600.0,2007-08-14,20
1,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,None,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
2,14,Altes,R.D.,None,8600 Moody Road,None,Fort Smith,AR,72903,2300.0,2007-06-21,16
3,16,Anthony,John,None,211 Long Island Drive,None,Hot Springs,AR,71913,2300.0,2007-06-12,16
4,22,Baker,David,None,2550 Adamsbrooke Drive,None,Conway,AR,72034,2300.0,2007-04-11,16


## SELECT-COLUMNS in SQL + Pandas

In [62]:
dfid[['amount', 'city']].head()

,amount,city
0,500.0,Floyd
1,250.0,Pleasanton
2,50.0,Pleasanton
3,100.0,Pleasanton
4,100.0,Bentonville


In [70]:
out = make_query("SELECT first_name, amount FROM contributors WHERE amount < 400")
make_frame(out, ['first_name', 'amount']).head()

,first_name,amount
0,Don,250.0
1,Don,50.0
2,Don,100.0
3,Charles,100.0
4,Brittni,250.0


## SELECT-DISTINCT in SQL + Pandas

In [78]:
dfid[['first_name', 'last_name']].count()

first_name    175
last_name     175
dtype: int64

In [77]:
dfid[['first_name', 'last_name']].drop_duplicates().count()

first_name    126
last_name     126
dtype: int64

In [93]:
out = make_query("SELECT DISTINCT last_name, first_name FROM contributors")
make_frame(out, ['last_name', 'first_name']).count()

last_name     126
first_name    126
dtype: int64

## ASSIGN in SQL + Pandas

In [17]:
dfid['name'] = dfid['last_name'] + ', ' + dfid['first_name']
dfid.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven"
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"


In [97]:
dfid.assign(ucname = dfid.last_name + ":" + dfid.first_name).head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name,ucname
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven",Agee:Steven
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don",Ahrens:Don
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don",Ahrens:Don
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don",Ahrens:Don
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles",Akin:Charles


In [98]:
dfid.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven"
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"


#### Change name in Pandas + SQL

In [101]:
dfid.loc[dfid.state == 'VA', 'name']

0           Agee, Steven
27       Buckheit, Bruce
77      Ranganath, Anoop
88     Perreault, Louise
145      ABDELLA, THOMAS
Name: name, dtype: object

In [102]:
dfid.loc[dfid.state == 'VA', 'name'] = 'junk'

In [103]:
dfid[dfid.state == 'VA']

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.00,2007-06-30,16,junk
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20,junk
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32,junk
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32,junk
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35,junk


Add new columns in SQL

In [106]:
alt = 'ALTER TABLE contributors ADD COLUMN name'
db.cursor().execute(alt)

In [107]:
make_query('PRAGMA table_info(contributors)')

[(0, 'id', 'INTEGER', 1, None, 1),
 (1, 'last_name', 'VARCHAR', 0, None, 0),
 (2, 'first_name', 'VARCHAR', 0, None, 0),
 (3, 'middle_name', 'VARCHAR', 0, None, 0),
 (4, 'street_1', 'VARCHAR', 0, None, 0),
 (5, 'street_2', 'VARCHAR', 0, None, 0),
 (6, 'city', 'VARCHAR', 0, None, 0),
 (7, 'state', 'VARCHAR', 0, None, 0),
 (8, 'zip', 'VARCHAR', 0, None, 0),
 (9, 'amount', 'INTEGER', 0, None, 0),
 (10, 'date', 'DATETIME', 0, None, 0),
 (11, 'candidate_id', 'INTEGER', 1, None, 0),
 (12, 'name', '', 0, None, 0)]

In [116]:
out = make_query('SELECT id, last_name, first_name FROM contributors')
out2 = [(e[1] + ', ' + e[2], e[0]) for e in out]
out2[0]

('Agee, Steven', 1)

In [120]:
alt2 = 'UPDATE contributors SET name = ? WHERE id = ?'
for line in out2:
    db.cursor().execute(alt2, line)
db.commit()

In [124]:
out = make_query('SELECT * FROM contributors')
make_frame(out, cont_cols + ['name']).head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven"
1,2,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,3,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,4,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,5,Akin,Charles,None,10187 Sugar Creek Road,None,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"


Assignment with existing columns

In [11]:
alt = "UPDATE contributors SET name = 'junk' WHERE state = 'VA';"
db.cursor().execute(alt)
db.commit()

In [14]:
out = make_query("SELECT * FROM contributors WHERE state = 'VA';")
make_frame(out, cont_cols)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500.00,2007-06-30,16,junk
1,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20,junk
2,78,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32,junk
3,89,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32,junk
4,146,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,None,FAIRFAX,VA,220308440,50.00,2007-09-30,35,junk


In [13]:
out = make_query('SELECT * FROM contributors')
make_frame(out).head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500.0,2007-06-30,16,junk
1,2,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,3,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,4,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,5,Akin,Charles,None,10187 Sugar Creek Road,None,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"


##  No Drop Column in SQLITE

In [15]:
alt = "ALTER TABLE contributors DROP COLUMN name;"
db.cursor().execute(alt)
db.commit()

OperationalError: near "DROP": syntax error

In [18]:
del dfid['name']

## AGGREGATE in SQL + Pandas

In [20]:
dfid.describe()

,zip,amount,candidate_id
count,1.750000e+02,175.000000,175.000000
mean,3.780014e+08,3.418114,28.000000
std,3.628278e+08,1028.418999,7.823484
min,2.474000e+03,-2592.000000,16.000000
25%,9.336700e+04,-175.000000,20.000000
50%,3.233313e+08,100.000000,32.000000
75%,7.816946e+08,300.000000,35.000000
max,9.951532e+08,4600.000000,37.000000


In [21]:
dfid['amount'].max()

4600.0

In [23]:
dfid[dfid['amount'] == dfid['amount'].max()]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.0,2007-08-14,20


In [26]:
out = make_query("SELECT *, MAX(amount) AS maxamt FROM contributors")
print(out)
make_frame(out, cont_cols + ['maxamt'])

[(31, 'Buckel', 'Linda', None, 'PO Box 683130', None, 'Park City', 'UT', '840683130', 4600, '2007-08-14', 20, 'Buckel, Linda', 4600)]


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name,maxamt
0,31,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,4600,2007-08-14,20,"Buckel, Linda",4600


In [36]:
out = make_query("SELECT COUNT(amount) AS AMOUNTCOUNT FROM contributors")
print(out)

[(175,)]


In [38]:
out = make_query("SELECT AVG(amount) FROM contributors")
print(out)

[(3.418114285714276,)]


In [39]:
dfid[dfid['amount'] > dfid['amount'].max() - 2300]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.0,2007-08-14,20
159,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,NaN,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37


In [41]:
out = make_query("SELECT * FROM contributors WHERE amount > (select (MAX(amount) - 2300) FROM contributors)")
make_frame(out)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,31,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,4600,2007-08-14,20,"Buckel, Linda"
1,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,None,MIAMI SPRINGS,FL,331663832,2600,2008-01-25,37,"ABATE, MARIA"


## GROUP-AGG in SQL + Pandas

In [45]:
dfid.groupby('state').sum()

,zip,amount,candidate_id
state,,,
AK,2985459621,1210.00,111
AR,864790,14200.00,192
AZ,860011121,120.00,37
CA,14736360720,-5013.73,600
CO,2405477834,-5823.00,111
CT,68901376,2300.00,35
DC,800341853,-1549.91,102
FL,8970626520,-4050.00,803
IA,50266,250.00,16


In [46]:
dfid.groupby('state')['amount'].mean()

state
AK     403.333333
AR    1183.333333
AZ     120.000000
CA    -217.988261
CO   -1455.750000
CT    2300.000000
DC    -309.982000
FL    -135.000000
IA     250.000000
ID    -261.000000
IL    -931.133333
KS    -330.000000
KY    -200.000000
LA     650.000000
MA     -13.833333
MD     150.000000
ME     630.000000
MI    -253.000000
MN     107.333333
MO     100.000000
NC     500.000000
NH     -24.600000
NJ    -408.725000
NV     181.250000
NY    -809.312500
OH     112.500000
OK     266.666667
PA    -429.200000
RI     100.000000
SC     800.000000
TN     -25.000000
TX     220.582222
UT     459.090909
VA     103.184000
WA    -166.666667
Name: amount, dtype: float64

In [47]:
dfid['state'].unique()

array(['VA', 'CA', 'AR', 'DC', 'SC', 'IA', 'OH', 'NC', 'UT', 'MO', 'IL',
       'ME', 'FL', 'MD', 'MI', 'CO', 'WA', 'NY', 'TX', 'KY', 'PA', 'TN',
       'MA', 'MN', 'KS', 'NJ', 'NH', 'ID', 'OK', nan, 'NV', 'CT', 'RI',
       'AK', 'LA', 'AZ'], dtype=object)

In [56]:
out = make_query("SELECT state, SUM(amount) FROM contributors GROUP BY state")
make_frame(out, legend = ['state', 'amount'])

,state,amount
0,None,-500.00
1,AK,1210.00
2,AR,14200.00
3,AZ,120.00
4,CA,-5013.73
5,CO,-5823.00
6,CT,2300.00
7,DC,-1549.91
8,FL,-4050.00
9,IA,250.00


### DELETE in SQL + Pandas

In [67]:
dfid2 = dfid.copy()
dfid2.set_index('last_name', inplace = True)
dfid2.head()

,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
last_name,,,,,,,,,,
Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


In [68]:
dfid2.drop('Ahrens', inplace = True)
dfid2.head()

,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
last_name,,,,,,,,,,
Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16
Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16


In [69]:
dfid2.reset_index(inplace = True)
dfid2.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
2,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
3,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16
4,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16


In [4]:
drop = "DELETE FROM contributors WHERE last_name = \"Ahrens\";"
db.cursor().execute(drop)

In [11]:
db.commit()
out = make_query("SELECT * FROM contributors")
make_frame(out).head(10)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500.0,2007-06-30,16,junk
1,5,Akin,Charles,None,10187 Sugar Creek Road,None,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"
2,6,Akin,Mike,None,181 Baywood Lane,None,Monticello,AR,71655,1500.0,2007-05-18,16,"Akin, Mike"
3,7,Akin,Rebecca,None,181 Baywood Lane,None,Monticello,AR,71655,500.0,2007-05-18,16,"Akin, Rebecca"
4,8,Aldridge,Brittni,None,"808 Capitol Square Place, SW",None,Washington,DC,20024,250.0,2007-06-06,16,"Aldridge, Brittni"
5,9,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1000.0,2007-06-11,16,"Allen, John D."
6,10,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1300.0,2007-06-29,16,"Allen, John D."
7,11,Allison,John W.,None,P.O. Box 1089,None,Conway,AR,72033,1000.0,2007-05-18,16,"Allison, John W."
8,12,Allison,Rebecca,None,3206 Summit Court,None,Little Rock,AR,72227,1000.0,2007-04-25,16,"Allison, Rebecca"
9,13,Allison,Rebecca,None,3206 Summit Court,None,Little Rock,AR,72227,200.0,2007-06-12,16,"Allison, Rebecca"


## Limit

In [12]:
out = make_query("SELECT * FROM contributors LIMIT 3")
make_frame(out)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500,2007-06-30,16,junk
1,5,Akin,Charles,None,10187 Sugar Creek Road,None,Bentonville,AR,72712,100,2007-06-16,16,"Akin, Charles"
2,6,Akin,Mike,None,181 Baywood Lane,None,Monticello,AR,71655,1500,2007-05-18,16,"Akin, Mike"


In [15]:
dfid[0:3]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16


## Indexes

In [25]:
crind = "CREATE INDEX amount_ix ON contributors(amount)"
db.cursor().execute(crind)
db.commit()

In [35]:
crind="DROP INDEX amount_ix;"
db.cursor().execute(crind)
db.commit()

## Simple subselect

In [36]:
dfid.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


In [49]:
obamaid = dfcand[dfcand.last_name == 'Obama']['id'].values[0]

In [54]:
obamacontri = dfid.query('candidate_id == %s' % obamaid)
obamacontri.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
25,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,50.0,2007-07-30,20
26,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,25.0,2007-08-16,20
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.0,2007-09-19,20
28,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,2300.0,2007-08-14,20
29,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,-2300.0,2007-08-14,20


In [56]:
select = """SELECT * FROM contributors WHERE 
    candidate_id = (SELECT id FROM candidates WHERE last_name = 'Obama')"""
out = make_query(select)
make_frame(out).head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,26,Buckler,Steve,None,24351 Armada Dr,None,Dana Point,CA,926291306,50.0,2007-07-30,20
1,27,Buckler,Steve,None,24351 Armada Dr,None,Dana Point,CA,926291306,25.0,2007-08-16,20
2,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.0,2007-09-19,20
3,29,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,2300.0,2007-08-14,20
4,30,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,-2300.0,2007-08-14,20


## Implicit Join

Including only Obama contributors

In [68]:
joinsel = """
SELECT
    contributors.last_name, contributors.first_name, contributors.amount, candidates.last_name
FROM
    contributors, candidates
WHERE contributors.candidate_id = candidates.id AND
    candidates.last_name = 'Obama'
"""

out = make_query(joinsel)
make_frame(out, legend = ['contributors.last_name', 'contributors.first_name', 'contributors.amount', 'candidates.last_name']).head()

,contributors.last_name,contributors.first_name,contributors.amount,candidates.last_name
0,Buckler,Steve,50.0,Obama
1,Buckler,Steve,25.0,Obama
2,Buckheit,Bruce,100.0,Obama
3,Buckel,Linda,2300.0,Obama
4,Buckel,Linda,-2300.0,Obama


Include not just Obama contributors

In [71]:
implicitjoinsel="""
SELECT 
    contributors.last_name, contributors.first_name, contributors.amount, candidates.last_name 
FROM 
    contributors, candidates 
WHERE contributors.candidate_id = candidates.id;
"""
out=make_query(implicitjoinsel)
make_frame(out, legend=["contributors.last_name", 
            "contributors.first_name", "contributors.amount", "candidates.last_name"]).head()

,contributors.last_name,contributors.first_name,contributors.amount,candidates.last_name
0,Agee,Steven,500.0,Huckabee
1,Ahrens,Don,250.0,Huckabee
2,Ahrens,Don,50.0,Huckabee
3,Ahrens,Don,100.0,Huckabee
4,Akin,Charles,100.0,Huckabee


## Explicit INNER JOIN (Pandas + SQL)

In [18]:
cols_wanted=['last_name_x', 'first_name_x', 'candidate_id', 'id', 'last_name_y']
dfid.merge(dfcand, left_on = 'candidate_id', right_on = 'id')[cols_wanted]

,last_name_x,first_name_x,candidate_id,id,last_name_y
0,Agee,Steven,16,16,Huckabee
1,Ahrens,Don,16,16,Huckabee
2,Ahrens,Don,16,16,Huckabee
3,Ahrens,Don,16,16,Huckabee
4,Akin,Charles,16,16,Huckabee
5,Akin,Mike,16,16,Huckabee
6,Akin,Rebecca,16,16,Huckabee
7,Aldridge,Brittni,16,16,Huckabee
8,Allen,John D.,16,16,Huckabee
9,Allen,John D.,16,16,Huckabee


In [29]:
joinsel = """
SELECT
    contributors.last_name, contributors.first_name, candidates.last_name
FROM
    contributors JOIN candidates
ON
    contributors.candidate_id = candidates.id
"""

out = make_query(joinsel)
make_frame(out, legend = ['contributors.last_name', 'contributors.first_name', 'candidates.last_name']).head()

,contributors.last_name,contributors.first_name,candidates.last_name
0,Agee,Steven,Huckabee
1,Ahrens,Don,Huckabee
2,Ahrens,Don,Huckabee
3,Ahrens,Don,Huckabee
4,Akin,Charles,Huckabee


In [33]:
joinsel = """
SELECT
    COUNT(contributors.id), contributors.last_name, candidates.last_name
FROM 
    contributors JOIN candidates
ON
    contributors.candidate_id = candidates.id
GROUP BY
    candidates.last_name
"""

out = make_query(joinsel)
make_frame(out, legend = ['contributors.id.count', 'contributors.last_name', 'candidates.last_name'])

,contributors.id.count,contributors.last_name,candidates.last_name
0,25,Aanonsen,Clinton
1,25,BROWN,Giuliani
2,25,Barbee,Huckabee
3,25,ABRAHAM,McCain
4,25,Harrison,Obama
5,25,Jacobs,Paul
6,25,ABBOTT,Romney


In [37]:
len(make_query("SELECT DISTINCT id, last_name FROM candidates"))

17

## Outer JOIN

left outer (contributors on candidates)

In [40]:
dfid.merge(dfcand, left_on = "candidate_id", right_on = 'id', how = 'left')[cols_wanted]

,last_name_x,first_name_x,candidate_id,id,last_name_y
0,Agee,Steven,16,16,Huckabee
1,Ahrens,Don,16,16,Huckabee
2,Ahrens,Don,16,16,Huckabee
3,Ahrens,Don,16,16,Huckabee
4,Akin,Charles,16,16,Huckabee
5,Akin,Mike,16,16,Huckabee
6,Akin,Rebecca,16,16,Huckabee
7,Aldridge,Brittni,16,16,Huckabee
8,Allen,John D.,16,16,Huckabee
9,Allen,John D.,16,16,Huckabee


In [41]:
joinsel = """
SELECT
    COUNT(contributors.id), contributors.first_name, candidates.last_name,
    contributors.candidate_id, candidates.id
FROM
    contributors LEFT OUTER JOIN candidates
ON 
    contributors.candidate_id = candidates.id
GROUP BY
    candidates.last_name
"""
out = make_query(joinsel)
make_frame(out, legend = ['COUNT(contributors.id)', 'contributors.first_name', 'candidates.last_name',
    'contributors.candidate_id', 'candidates.id'])

,COUNT(contributors.id),contributors.first_name,candidates.last_name,contributors.candidate_id,candidates.id
0,25,Lin,Clinton,34,34
1,25,TIMOTHY,Giuliani,22,22
2,25,John,Huckabee,16,16
3,25,SALEM,McCain,37,37
4,25,Ryan,Obama,20,20
5,25,Richard,Paul,32,32
6,25,GERALD,Romney,35,35


### Right outer (contributors on candidates) = left outer (candidates on contributors)

In [47]:
dfid.merge(dfcand, left_on = 'candidate_id', right_on = 'id', how = 'right')[cols_wanted]

,last_name_x,first_name_x,candidate_id,id,last_name_y
0,Agee,Steven,16.0,16,Huckabee
1,Ahrens,Don,16.0,16,Huckabee
2,Ahrens,Don,16.0,16,Huckabee
3,Ahrens,Don,16.0,16,Huckabee
4,Akin,Charles,16.0,16,Huckabee
5,Akin,Mike,16.0,16,Huckabee
6,Akin,Rebecca,16.0,16,Huckabee
7,Aldridge,Brittni,16.0,16,Huckabee
8,Allen,John D.,16.0,16,Huckabee
9,Allen,John D.,16.0,16,Huckabee


In [49]:
joinsel="""
SELECT 
    COUNT(contributors.id), contributors.first_name, candidates.last_name, 
        contributors.candidate_id, candidates.id
FROM 
    candidates LEFT OUTER JOIN contributors 
ON 
    contributors.candidate_id = candidates.id
GROUP BY 
    candidates.last_name;
"""
out=make_query(joinsel)
make_frame(out, legend=["count(contributors.id)", "contributors.first_name",  
                    "candidates.last_name", "contributors.candidate_id", "candidates.id"])

,count(contributors.id),contributors.first_name,candidates.last_name,contributors.candidate_id,candidates.id
0,0,None,Biden,NaN,33
1,0,None,Brownback,NaN,36
2,25,Thomas,Clinton,34.0,34
3,0,None,Dodd,NaN,39
4,0,None,Edwards,NaN,26
5,25,WALTER,Giuliani,22.0,22
6,0,None,Gravel,NaN,24
7,25,William,Huckabee,16.0,16
8,0,None,Hunter,NaN,30
9,0,None,Kucinich,NaN,31


### Full outer (not support in SQL)

In [50]:
dfid.merge(dfcand, left_on = 'candidate_id', right_on = 'id', how = 'outer')[cols_wanted]

,last_name_x,first_name_x,candidate_id,id,last_name_y
0,Agee,Steven,16.0,16,Huckabee
1,Ahrens,Don,16.0,16,Huckabee
2,Ahrens,Don,16.0,16,Huckabee
3,Ahrens,Don,16.0,16,Huckabee
4,Akin,Charles,16.0,16,Huckabee
5,Akin,Mike,16.0,16,Huckabee
6,Akin,Rebecca,16.0,16,Huckabee
7,Aldridge,Brittni,16.0,16,Huckabee
8,Allen,John D.,16.0,16,Huckabee
9,Allen,John D.,16.0,16,Huckabee


## Pandas read SQL 

In [51]:
pd.read_sql("SELECT * FROM candidates WHERE party= 'D';", db)

,id,first_name,last_name,middle_name,party
0,20,Barack,Obama,None,D
1,24,Mike,Gravel,None,D
2,26,John,Edwards,None,D
3,29,Bill,Richardson,None,D
4,31,Dennis,Kucinich,None,D
5,33,Joseph,Biden,None,D
6,34,Hillary,Clinton,R.,D
7,39,Christopher,Dodd,J.,D


In [53]:
joinsel = """
SELECT
    contributors.last_name, contributors.first_name, candidates.last_name
FROM
    contributors JOIN candidates
ON
    contributors.candidate_id = candidates.id
"""
pd.read_sql(joinsel, db)

,last_name,first_name,last_name
0,Agee,Steven,Huckabee
1,Ahrens,Don,Huckabee
2,Ahrens,Don,Huckabee
3,Ahrens,Don,Huckabee
4,Akin,Charles,Huckabee
5,Akin,Mike,Huckabee
6,Akin,Rebecca,Huckabee
7,Aldridge,Brittni,Huckabee
8,Allen,John D.,Huckabee
9,Allen,John D.,Huckabee
